**Урок 2. Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)**

**Домашнее задание**

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

## Выполнение

### Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана.  Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [1]:
import pandas as pd

#### Загрузим новости, пользователей пользователей и списки последних прочитанных новостей

In [2]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")

In [3]:
display(news.head(3), users.head(3))

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

#### Получаем векторные представления новостей

In [149]:
"Gensim — библиотека обработки естественного языка предназначения для «Тематического моделирования»."
%pip install --upgrade gensim

"""
razdel - rule-based system for Russian sentence and word tokenization. See natasha.github.io article for more info.
https://github.com/natasha/razdel"""
%pip install razdel

"Морфологический анализатор pymorphy2"
%pip install pymorphy2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.2 MB 329 kB/s eta 0:00:01
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
#предобработка текстов
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from razdel import tokenize  # https://github.com/natasha/razdel
import pymorphy2

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stopword_ru = nltk.corpus.stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [6]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:800: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


In [7]:
def clean_text(text):
    """
    Очистка текста
    :return: Очищеный текст
    """
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '',
                  text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    # tokens = list(tokenize(text))
    # words = [_.text for _ in tokens]
    # words = [w for w in words if w not in stopword_ru]

    # return " ".join(words)
    return text


cache = {}


def lemmatization(text):
    """
    Лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова
    :return: Список отлемматизированых токенов
    """
    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-':  # [2]
            w = w[1:]
        if len(w) > 1:  # [3]
            if w in cache:  # [4]
                words_lem.append(cache[w])
            else:  # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [
        i for i in words_lem if not i in stopword_ru
    ]  # [6]

    return words_lem_without_stopwords

In [12]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:14: FutureWarning: Possible nested set at position 39
  from collections import abc


CPU times: user 21.4 s, sys: 1.27 s, total: 22.6 s
Wall time: 24.8 s


In [14]:
#!M
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 2min 56s, sys: 1.34 s, total: 2min 57s
Wall time: 2min 59s


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 325752
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}


#### А теперь в 3 строчки обучим нашу модель

In [15]:
#!M
%%time
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

CPU times: user 14.5 s, sys: 1.16 s, total: 15.6 s
Wall time: 21.1 s


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 27000
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 27000
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to seri

Что такое common_dictionary и как он выглядит

In [16]:
print([common_dictionary[i] for i in range(10, 15)])

['ватутин', 'взаимодействие', 'власть', 'войти', 'вячеслав']


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

Все просто - это словарь наших слов

Запускаем обучение

In [17]:
#!M
%%time
from gensim.models import LdaModel

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary, passes=5)

CPU times: user 5min 10s, sys: 8min 34s, total: 13min 44s
Wall time: 2min 39s


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

In [18]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [19]:
# Create a new corpus, made of previously unseen documents.
# Создайте новый корпус, состоящий из ранее невидимых документов.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]

print(other_texts[2])
lda[unseen_doc]

['форвард' 'авангард' 'томаш' 'заборский' 'прокомментировать' 'игра'
 'команда' 'матч' 'чемпионат' 'кхл' 'против' 'атланта' 'nnnn' 'плохой'
 'матч' 'нижний' 'новгород' 'против' 'торпедо' 'настраиваться' 'первый'
 'минута' 'включиться' 'заборский' 'получиться' 'забросить' 'быстрый'
 'гол' 'задать' 'хороший' 'темп' 'поединок' 'играть' 'хороший' 'сторона'
 'пять' 'очко' 'выезд' 'девять' 'хороший']


[(3, 0.06257276),
 (7, 0.15017986),
 (11, 0.09844572),
 (15, 0.033536147),
 (17, 0.492036),
 (22, 0.099585906),
 (23, 0.04508121)]

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

In [20]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: ракета система первый земля станция новый скорость
topic_1: сша военный россия российский американский сила армия
topic_2: школа русский франция язык французский париж общество
topic_3: тыс рост млн составить цена россия рубль
topic_4: москва nn день мероприятие центр столица московский
topic_5: пенсия журнал продукция возраст звезда писать популярный
topic_6: гражданин закон решение документ право уголовный россия
topic_7: исследование млрд фонд технология снижение эксперимент млн
topic_8: мышь медицина орден выдавать приток аргентина фунт
topic_9: космос воздух пилот лодка атмосферный лётчик небо
topic_10: nn произойти двигатель запуск экипаж причина аэропорт
topic_11: новый проект развитие научный россия эксперт российский
topic_12: nn ребёнок женщина мужчина статья nnn газета
topic_13: россия украина российский глава газ украинский санкция
topic_14: банк nn рубль директор дмитрий владимир александр
topic_15: помощь день врач рак лечение организм препарат
topic_16: земля см

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [21]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(
        zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

In [22]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.032223,0.0,0.000000,0.0,0.000000,0.099495,0.000000,0.0,...,0.000000,0.0,0.103344,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,4896,0.000000,0.000000,0.0,0.252549,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.339889,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,4897,0.000000,0.000000,0.0,0.062509,0.0,0.000000,0.000000,0.150183,0.0,...,0.033535,0.0,0.492042,0.0,0.0,0.0,0.0,0.099616,0.045084,0.0
3,4898,0.166821,0.000000,0.0,0.000000,0.0,0.039298,0.000000,0.000000,0.0,...,0.000000,0.0,0.236896,0.0,0.0,0.0,0.0,0.389356,0.000000,0.0
4,4899,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.155859,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 141166
Use %enable_full_walk to se

Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

In [23]:
import warnings

warnings.filterwarnings('ignore')

### Следующий шаг - векторные представления пользователей

In [ ]:
# users.head(3)

In [24]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [ ]:
# doc_dict[293622]

In [25]:
user_articles_list = users['articles'].iloc[33]
display(user_articles_list, 
        eval(user_articles_list)
       )

'[323329, 321961, 324743, 323186, 324632, 474690]'

[323329, 321961, 324743, 323186, 324632, 474690]

---

In [26]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [27]:
def score_with_user_embedding(func):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: func(x), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] + [f'topic_{i}' for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(
    X[[f'topic_{i}' for i in range(25)]], X['churn'], random_state=0)
    
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)

    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    roc_auc_score = roc_auc_score(y_test, preds)

    return round(thresholds[ix], 4), round(fscore[ix], 4), round(precision[ix], 4), round(recall[ix], 4), round(roc_auc_score, 4)

In [28]:
target = pd.read_csv("users_churn.csv")

### Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: `mean`, `median`, `max`, `idf_mean` по метрикам `roc_auc`, `precision`, `recall`, `f_score`

In [29]:
results = pd.DataFrame(np.array([
    score_with_user_embedding(func=get_user_embedding_mean),
    score_with_user_embedding(func=get_user_embedding_median),
    score_with_user_embedding(func=get_user_embedding_max)
]), columns=['Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['func'] = ['mean', 'median', 'max']
results = results.set_index('func')

In [30]:
results

,Best Threshold,F-Score,Precision,Recall,ROC AUC score
func,,,,,
mean,0.2575,0.6955,0.6446,0.7551,0.9473
median,0.2598,0.7455,0.6645,0.8490,0.9626
max,0.3536,0.7265,0.7265,0.7265,0.9569


Судя по Precision, самый эффективный способ получается через `max`. 